# Evaluating Model Fitness

## Detecting Overfitting and Underfitting

## Objectives
* Compare training and testing scores to detect overfit/underfit models
* Use best practices to reduce overfitting in models
* Use best practices to reduce underfitting in models

## Evaluating Model Fitness

In this lab, we're going to build a classifier on a small subset of the titanic dataset.  We'll make use of training and testing validation methods that we learned in previous labs to detect when our model is overfit or underfit.  We'll then implement strategies to fix our overfit and underfit models.  

We'll start by importing all the libraries we'll need, and reading in the Titanic Dataset.  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

